## Unsupervised Anomaly Detection
: Before taking semi-supervised learning(SSL), using classical(*shallow*) methods for unsupervised model(One-class SVM and IsolationForest)

In [1]:
# # for colab
# from google.colab import drive
# drive.mount('/content/Mydrive')
# %cd "/content/Mydrive/MyDrive/Github/dacon_FraudDetection"

Drive already mounted at /content/Mydrive; to attempt to forcibly remount, call drive.mount("/content/Mydrive", force_remount=True).
/content/Mydrive/MyDrive/Github/dacon_FraudDetection


In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [3]:
# Data Load
df_train = pd.read_csv("dataset/train.csv", index_col = 0)
df_val = pd.read_csv("dataset/val.csv", index_col = 0)
df_test = pd.read_csv("dataset/test.csv", index_col = 0)
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(113842, 30)
(28462, 31)
(142503, 30)


In [4]:
# StandardScaler
scaler = StandardScaler()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
train = scaler.fit_transform(df_train)
valid = scaler.fit_transform(df_val.iloc[:,:30])
valid_label = df_val['Class']
# label 변환
valid_label = valid_label.replace(1, -1)
valid_label = valid_label.replace(0, 1)
valid_label.value_counts() # normal : 1, anomaly : -1

 1    28432
-1       30
Name: Class, dtype: int64

### Gridsearch with each model
기존 GridSearchCV 모듈은 unsupervised setting에 적합하지 않으므로(Cross validation이 불가능하므로) labeled data를 이용하여 f1_score을 계산하는 방식 고안 필요

In [8]:
# gridsearch
import itertools
from sklearn.metrics import f1_score

def gridsearch(model, param_grid, x_test, y_test):
    '''
    model : scikit-learn model name(str),
    param_grid : dict form of grid parameters
    x_test, y_test : scoring data and label
    '''
    keys, values = zip(*param_grid.items())
    permutations_dicts = [dict(zip(keys,v)) for v in itertools.product(*values)]
    
    res = [] # 결과 저장할 double list
    for i in range(len(permutations_dicts)):
      param = permutations_dicts[i]
      model_p = globals()[model](**param)
      model_p.fit(train)
      predict_label = model_p.predict(x_test)
      score = f1_score(y_true=y_test, y_pred=predict_label, average='macro') # macro f1 score
      res.append(list(param.values())+[score.round(3)])
      print(f"{i}th search completed!")

    df = pd.DataFrame(res, columns=list(param_grid.keys())+['score']) # to dataframe
    
    return df # return result dataframe

In [9]:
# # OneClassSVM >> TOOO SLOW
# param_grid = {
#     'kernel' : ['linear','rbf'],
#     'tol' : [0.001,0.01,0.05,0.1,0.2], # 0.2 for pollution rate    
#     'verbose' : [10]
# }
# res_ocsvm = gridsearch('OneClassSVM', param_grid, valid, valid_label)
# res_ocsvm

In [ ]:
# IsolationForest
param_grid = {
    'contamination' : np.logspace(-1,-5,num = 9),
    'verbose' : [1]
}
res_forest = gridsearch('IsolationForest', param_grid, valid, valid_label)

In [29]:
res_forest.to_csv('res_forest.csv')

In [ ]:
ocsvm = OneClassSVM(verbose = 10, kernel = 'rbf', tol = 0.001)
ocsvm.fit(train)

[LibSVM]